#PREPARAR Y ENTRENAR EL MODELO

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

Para entrenar el modelo voy a utilizar el conjunto de datos cifar10 el cual ya está incluído en tf keras, este conjunto de datos es bastante completo y tiene subconjuntos de entrenamiento y test, pero vamos a realizar nosotros el test manualmente.

In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()

170498071/170498071 [==============================] - 6s 0us/step


Voy a distinguir los siguientes objetos: gatos(3), ciervos(4), perros(5) y caballos (7)



In [3]:
clases_objetos = [3, 4, 5, 7]
nombres_objetos={3:"gato",4:"ciervo",5:"perro",7:"caballo"}
filtro_train = np.isin(train_labels, clases_objetos).flatten()
filtro_test = np.isin(test_labels, clases_objetos).flatten()
filtro_test.shape

(10000,)

In [4]:
train_images, train_labels = train_images[filtro_train], train_labels[filtro_train]
test_images, test_labels = test_images[filtro_test], test_labels[filtro_test]

Pasar a categórica

In [5]:
train_labels = tf.keras.utils.to_categorical(np.array([clases_objetos.index(x) for x in train_labels]), num_classes=4)
test_labels = tf.keras.utils.to_categorical(np.array([clases_objetos.index(x) for x in test_labels]), num_classes=4)

Normalizar

In [6]:
train_images, test_images = train_images / 255.0, test_images / 255.0

Aquí defino mi modelo Sequential, he preparado la primera capa convolucional para que acepte imagenes de 32x32 con RGB ya que he considerado que para identificar distintos mamíferos el color es bastante importante

El modelo está formado principalmente por capas convolucionales, que van de menos kernels a más, de esta manera las capas finales van a ser capaces de detectar más características en las imágenes.

Además de las capas Dense, MaxPooling2D y Conv2D también he incluído capas BatchNormalization y Dropout, ambas sirven para evitar el overfitting, la primera capa normaliza los tensores de las capas Conv2D y Dropout directamente descarta de una parte de los tensores.

In [7]:
modelo = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(4, activation='softmax')
])
modelo.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 batch_normalization (BatchN  (None, 30, 30, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 32)        9248      
                                                                 
 batch_normalization_1 (Batc  (None, 28, 28, 32)       128       
 hNormalization)                                                 
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                        

In [8]:
optimizador = tf.keras.optimizers.Adam(learning_rate=0.001)
modelo.compile(optimizer=optimizador,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [9]:
historial = modelo.fit(train_images, train_labels, epochs=500, batch_size=32, validation_split=0.2)


Epoch 1/500
500/500 [==============================] - 18s 9ms/step - loss: 1.4503 - accuracy: 0.4198 - val_loss: 1.1974 - val_accuracy: 0.4692
Epoch 2/500
500/500 [==============================] - 5s 10ms/step - loss: 1.0650 - accuracy: 0.5412 - val_loss: 0.9250 - val_accuracy: 0.6030
Epoch 3/500
500/500 [==============================] - 4s 9ms/step - loss: 0.9471 - accuracy: 0.5945 - val_loss: 1.0424 - val_accuracy: 0.5573
Epoch 4/500
500/500 [==============================] - 4s 8ms/step - loss: 0.8999 - accuracy: 0.6246 - val_loss: 0.8507 - val_accuracy: 0.6463
Epoch 5/500
500/500 [==============================] - 5s 10ms/step - loss: 0.8325 - accuracy: 0.6561 - val_loss: 0.7942 - val_accuracy: 0.6790
Epoch 6/500
500/500 [==============================] - 4s 9ms/step - loss: 0.7845 - accuracy: 0.6822 - val_loss: 0.8093 - val_accuracy: 0.6610
Epoch 7/500
500/500 [==============================] - 4s 9ms/step - loss: 0.7430 - accuracy: 0.7038 - val_loss: 0.7003 - val_accuracy: 0.7

Guardar el modelo por si acaso

In [10]:
modelo.save('cnn_500.h5')

#PROBAR EL MODELO

Aquí he programado una función simple para preparar las imágenes, de esta manera para probar las imagénes rápido basta con cambiar el nombre del archivo que esté cargado en el cuaderno

Voy a adjuntar varias imágenes que he probado yo y me las predice con 100% de eficacia, aunque es aconsejable probar un par de imágenes nuevas que encuentres en internet

In [11]:
def preparar_imagen(img):
  img_redimensionada = img.resize((32, 32))
  img_array = np.array(img_redimensionada) / 255.0
  return np.expand_dims(img_array, axis=0)

Cambiar **aquí** el nombre del archivo

In [85]:
img=Image.open('gato3.jpg')

In [86]:
img_preparada=preparar_imagen(img)

In [87]:
prediccion=modelo.predict(img_preparada)

1/1 [==============================] - 0s 19ms/step


In [88]:
objeto_prediccion=np.argmax(prediccion)

Predicción:

In [89]:
nombres_objetos[clases_objetos[objeto_prediccion]]

'gato'